In [1]:
%pylab inline
from ipyparallel import Client, error
cluster=Client(profile="mpi") 
view=cluster[:]
view.block=True

try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

Populating the interactive namespace from numpy and matplotlib


```{note}
This feature requires MPI, and may not be able to be run on Colab.
```

# Adding Constraints

To add a constraint to an optimization, use the `add_constraint` method on System.

```{eval-rst}
    .. autoclass:: openmdao.core.system.System.add_constraint
        :noindex:
```

## Specifying units

You can specify units when adding a constraint. When this is done, the constraint value is converted from the target output’s units to the desired unit before giving it to the optimizer. If you also specify scaling, that scaling is applied after the unit conversion. Moreover, the upper and lower limits in the constraint definition should be specified using these units.



In [2]:
import openmdao.api as om

prob = om.Problem()
model = prob.model

model.add_subsystem('comp1', om.ExecComp('y1 = 2.0*x',
                                         x={'value': 2.0, 'units': 'degF'},
                                         y1={'value': 2.0, 'units': 'degF'}),
                    promotes=['x', 'y1'])

model.add_subsystem('comp2', om.ExecComp('y2 = 3.0*x',
                                         x={'value': 2.0, 'units': 'degF'},
                                         y2={'value': 2.0, 'units': 'degF'}),
                    promotes=['x', 'y2'])

model.set_input_defaults('x', 35.0, units='degF')

model.add_design_var('x', units='degC', lower=0.0, upper=100.0)
model.add_constraint('y1', units='degC', lower=0.0, upper=100.0)
model.add_objective('y2', units='degC')

prob.setup()
prob.run_driver()

False

In [3]:
print('Model variables')
print(prob.get_val('x', indices=[0]))

Model variables
[35.]


In [4]:
print(prob.get_val('comp2.y2', indices=[0]))

[105.]


In [5]:
print(prob.get_val('comp1.y1', indices=[0]))

[70.]


In [6]:
print('Driver variables')
dv = prob.driver.get_design_var_values()
print(dv['x'][0])

Driver variables
1.6666666666666983


In [7]:
obj = prob.driver.get_objective_values(driver_scaling=True)
print(obj['comp2.y2'][0])

40.555555555555586


In [8]:
con = prob.driver.get_constraint_values(driver_scaling=True)
print(con['comp1.y1'][0])

21.111111111111143


In [9]:
import numpy as np
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('x', indices=[0]), 35.)
assert_near_equal(prob.get_val('comp2.y2', indices=[0]), 105.)
assert_near_equal(prob.get_val('comp1.y1', indices=[0]), 70.)
assert_near_equal(dv['x'][0], 1.6666666666666983)
assert_near_equal(obj['comp2.y2'][0], 40.555555555555586)
assert_near_equal(con['comp1.y1'][0], 21.111111111111143)

0.0

## Using the output of a distributed component as a constraint

You can use an output of a distributed component as a constraint or an objective. OpenMDAO automatically collects the values from all processors and provides them to the driver.

Here is an example where we perform optimization on a model that contains a `DistParabFeature` component that is distributed. The output is declared as a inequality constraint.

In [10]:
%%px

import numpy as np
import openmdao.api as om

from openmdao.test_suite.components.paraboloid_distributed import DistParabFeature

size = 7

prob = om.Problem()
model = prob.model

ivc = om.IndepVarComp()
ivc.add_output('x', np.ones((size, )))
ivc.add_output('y', -1.42 * np.ones((size, )))
ivc.add_output('offset', -3.0 + 0.6 * np.arange(size))

model.add_subsystem('p', ivc, promotes=['*'])
model.add_subsystem("parab", DistParabFeature(arr_size=size),
                    promotes=['*'])
model.add_subsystem('sum', om.ExecComp('f_sum = sum(f_xy)',
                                       f_sum=np.ones(1),
                                       f_xy=np.ones(size)),
                    promotes=['*'])

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_constraint('f_xy', lower=0.0)
model.add_objective('f_sum', index=-1)

prob.driver = om.pyOptSparseDriver(optimizer='SLSQP')
prob.setup(force_alloc_complex=True)

prob.run_driver()

[stdout:0] 

Optimization Problem -- Optimization using pyOpt_sparse
    Objective Function: _objfunc

    Solution: 
--------------------------------------------------------------------------------
    Total Time:                    0.0267
       User Objective Time :       0.0038
       User Sensitivity Time :     0.0145
       Interface Time :            0.0077
       Opt Solver Time:            0.0008
    Calls to Objective Function :       7
    Calls to Sens Function :            7


   Objectives
      Index  Name                 Value          Optimum
          0  sum.f_sum     1.150150E+01     0.000000E+00

   Variables (c - continuous, i - integer, d - discrete)
      Index  Name    Type      Lower Bound            Value      Upper Bound     Status
          0  p.x_0      c    -5.000000E+01     2.657527E+00     5.000000E+01           
          1  p.x_1      c    -5.000000E+01     2.604332E+00     5.000000E+01           
          2  p.x_2      c    -5.000000E+01     2.510060

[stderr:0] 
/Users/hschilli/Documents/OpenMDAO/dev/OpenMDAO/openmdao/core/component.py:522: DeprecationWarning:'parab' <class DistParabFeature>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.
/Users/hschilli/Documents/OpenMDAO/dev/OpenMDAO/openmdao/core/component.py:522: DeprecationWarning:'parab' <class DistParabFeature>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.
/Users/hschilli/Documents/OpenMDAO/dev/OpenMDAO/openmdao/core/component.py:522: DeprecationWarning:'parab' <class DistParabFeature>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.
[stderr:1] 
/Users/hschilli/Documents/OpenMDAO/dev/OpenMDAO/openmdao/core/component.p

Out[0:99]: False

Out[1:99]: False

In [11]:
%%px

desvar = prob.get_val('p.x', get_remote=True)
obj = prob.get_val('f_sum', get_remote=True)

print(desvar)

[stdout:0] 
[2.65752672 2.60433212 2.51005989 1.91021257 1.3100763  0.70992863
 0.10978096]
[stdout:1] 
[2.65752672 2.60433212 2.51005989 1.91021257 1.3100763  0.70992863
 0.10978096]


In [12]:
%%px

print(obj)

[stdout:0] [11.50150011]
[stdout:1] [11.50150011]


In [13]:
%%px

from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('p.x'), np.array([2.65752672, 2.60433212, 2.51005989, 1.91021257, 1.3100763,  0.70992863, 0.10978096]), 1e-6 )
assert_near_equal(prob.get_val('f_sum'),11.50150011, 1e-6 )

Out[0:102]: 2.0289233637542958e-10

Out[1:102]: 2.0289233637542958e-10